In [7]:
import Bio.PDB as PDB 
import numpy as np 
import freesasa
import glob

# Calculate parameters

In [10]:
surfaces = []
rsas = []
surface_seq = []

for file in glob.glob("data/training/crystal_structs/*.pdb"): 
    
    # parse the pdb file
    p = PDB.PDBParser(QUIET=True)
    s = p.get_structure(file, file) 
    
    # get the surface area
    structure = freesasa.Structure(file)
    result = freesasa.calc(structure)
    area_classes = freesasa.classifyResults(result, structure)
    
    # save this into numpy sheet result.totalArea() 
    surface = result.totalArea()
    surfaces.append(surface)
    
    # get the sequence length
    rsa = 0
    for chain in s.get_chains():
        rsa += len([_ for _ in chain.get_residues() if PDB.is_aa(_)])
    
    # save into numpy sheet rsa
    rsas.append(rsa)
    
    # caculate the surface/sequence 
    surface_seq.append(surface/rsa)

In [29]:
a = np.array(surfaces)
b = np.array(rsas)
c = np.array(surface_seq)

arr = np.column_stack((a, b, c))
np.savetxt("parameters.csv", arr, delimiter=",")